In [5]:
from google.colab import drive
drive.mount('/content/drive')

import joblib
import numpy as np
import pandas as pd

MODEL_PATH = "/content/drive/MyDrive/Final_Project_ML_OL/data/final_model.joblib"
PREP_PATH  = "/content/drive/MyDrive/Final_Project_ML_OL/data/preprocessor.joblib"

model = joblib.load(MODEL_PATH)
preprocessor = joblib.load(PREP_PATH)

num_features = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','TotalSpend','GroupSize']
cat_features = ['HomePlanet','Destination','CabinDeck','CabinSide','CryoSleep','VIP']

def prompt_bool(msg):
    return input(msg + " (y/n): ").strip().lower() in ["y", "yes", "true", "1"]

def split_cabin_single(cabin):
  # returns (CabinDeck, CabinSide) from "G/25/P" etc.
  if cabin is None or (isinstance(cabin, float) and np.isnan(cabin)) or str(cabin).strip() == "":
      return np.nan, np.nan
  parts = str(cabin).split("/")
  deck = parts[0] if len(parts) >= 1 else np.nan
  side = parts[2] if len(parts) >= 3 else np.nan
  return deck, side

def predict_one_from_prompts():
    print("\nEnter passenger info:")
    row = {
        "HomePlanet": input("HomePlanet (Earth/Mars/Europa or blank): ").strip() or np.nan,
        "CryoSleep":  prompt_bool("CryoSleep?"),
        "Cabin":      input("Cabin (e.g., deck/num/side or blank): ").strip() or np.nan,
        "Destination":input("Destination (TRAPPIST-1e/55 Cancri e/PSO J318.5-22 or blank): ").strip() or np.nan,
        "Age":        float(input("Age (number or blank): ").strip() or "nan"),
        "VIP":        prompt_bool("VIP?"),
        "RoomService":float(input("RoomService spend (number): ").strip() or "0"),
        "FoodCourt":  float(input("FoodCourt spend (number): ").strip() or "0"),
        "ShoppingMall":float(input("ShoppingMall spend (number): ").strip() or "0"),
        "Spa":        float(input("Spa spend (number): ").strip() or "0"),
        "VRDeck":     float(input("VRDeck spend (number): ").strip() or "0"),
        "Name":       input("Name (optional; blank ok): ").strip() or np.nan,
        "GroupSize":  int(input("GroupSize (enter a number, default 1): ").strip() or "1"),
    }

    cabin_deck, cabin_side = split_cabin_single(row["Cabin"])
    total_spend = row["RoomService"] + row["FoodCourt"] + row["ShoppingMall"] + row["Spa"] + row["VRDeck"]
    row["CabinDeck"] = cabin_deck
    row["CabinSide"] = cabin_side
    row["TotalSpend"] = total_spend

    X_new_raw = pd.DataFrame([row])[num_features + cat_features]
    X_new = preprocessor.transform(X_new_raw)  # matches your training pipeline

    pred = model.predict(X_new)[0]
    proba = model.predict_proba(X_new)[0] if hasattr(model, "predict_proba") else None

    label = "Transported" if bool(pred) else "Not Transported"
    print("\nPrediction:", label)

    if proba is not None and len(proba) == 2:
        print("Probability Transported:", float(proba[1]))

# Run it:
predict_one_from_prompts()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Enter passenger info:
HomePlanet (Earth/Mars/Europa or blank): Earth
CryoSleep? (y/n): y
Cabin (e.g., deck/num/side or blank): G/33/P
Destination (TRAPPIST-1e/55 Cancri e/PSO J318.5-22 or blank): 55 Cancri e
Age (number or blank): 30
VIP? (y/n): y
RoomService spend (number): 100
FoodCourt spend (number): 100
ShoppingMall spend (number): 100
Spa spend (number): 100
VRDeck spend (number): 100
Name (optional; blank ok): aidan waaler
GroupSize (enter a number, default 1): 2

Prediction: Not Transported
Probability Transported: 0.3592753646158881
